### Load libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [2]:
from dotenv import load_dotenv
import os

In [3]:
import pandas as pd
import numpy as np
import sys
import sqlite3

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

In [4]:
load_dotenv() # load environment variables

True

### Load Spotipy credentials

In [5]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Create search engine using Spotipy API

In [6]:
lookup = 'A lannister always pays his debts'# 'nite fleit'

In [7]:
results = sp.search(q=lookup, limit=10) #, type="track")

In [9]:
for i, track in enumerate(results['tracks']['items']):
    print(' ', i, track['name'],'-', track['artists'][0]['name'], '-', track['id'],
         '-', track['external_urls']['spotify'])

  0 A Lannister Always Pays His Debts - Ramin Djawadi - 1vPEPhmGDnp8RR6GCBozaa - https://open.spotify.com/track/1vPEPhmGDnp8RR6GCBozaa
  1 A Lannister Always Pays His Debts - Ramin Djawadi - 7bepfooXfeEsSK75XGo3W1 - https://open.spotify.com/track/7bepfooXfeEsSK75XGo3W1
  2 A Lannister Always Pays His Debts - The City of Prague Philharmonic Orchestra - 4VbVDcEsFp9lvmQXoLoui3 - https://open.spotify.com/track/4VbVDcEsFp9lvmQXoLoui3
  3 A Lannister Always Pays His Debts - Dominik Hauser - 6FAHZt535KpBIBa0YMEoZW - https://open.spotify.com/track/6FAHZt535KpBIBa0YMEoZW
  4 A Lannister Always Pays His Debts - Ramin Djawadi - 0euDS6hOtxuVP9tK0YGUKx - https://open.spotify.com/track/0euDS6hOtxuVP9tK0YGUKx
  5 A Lannister Always Pays His Debts - NDG - 7yJAoibVB0Ei2MLEi9ExjC - https://open.spotify.com/track/7yJAoibVB0Ei2MLEi9ExjC
  6 A Lannister Always Pays His Debts - TV Sounds Unlimited - 5enoGqbp3D0cZfzlx5EtUU - https://open.spotify.com/track/5enoGqbp3D0cZfzlx5EtUU
  7 A Lannister Always Pays Hi

In [43]:
song_id = '0gjfdXCV6yrwhP23NVx1Fm'

In [44]:
song_id

'0gjfdXCV6yrwhP23NVx1Fm'

In [45]:
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=A+lannister+always+pays+his+debts&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
       'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
       'id': '1hCkSJcXREhrodeIHQdav8',
       'name': 'Ramin Djawadi',
       'type': 'artist',
       'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
    

In [60]:
track_id='5enoGqbp3D0cZfzlx5EtUU'
rel_artists = sp.artist_related_artists(sp.track(track_id=track_id)['artists'][0]['id'])['artists']
artist_log = []
for a in rel_artists:
    artist_log.append(a['id'])
    feat_log = []
for artist in artist_log:
    for track in sp.artist_top_tracks(artist)['tracks']:
        feat_log.append(sp.audio_features(track['id'])[0])
        
        catalog = pd.DataFrame.from_dict(feat_log)
    
        root = pd.DataFrame.from_dict(sp.audio_features(tracks=[track_id]))

        merged_df = root.append(catalog, ignore_index=True)
    
        dropped_df = merged_df.drop(columns=['uri', 'track_href', 'id', 'duration_ms', 'time_signature', 'mode', 'loudness', 'type', 'analysis_url'])

In [61]:
dropped_df.shape

(116, 9)

In [189]:
def dj_rec(track_id, max_distance=6.5, neighbors=3):
    """
    Prints the ids of relevant songs, along with their distance from the input song.

    Parameters:
    track_id (string): Spotify track id.
    max_distance (float): maximum euclidean distance a song can be 
                          from the input song for it to be returned.
    neighbors (int): number of song recommendations returned.
    """

    rel_artists = sp.artist_related_artists(sp.track(track_id=track_id)['artists'][0]['id'])['artists']
    artist_log = []
    for a in rel_artists:
        artist_log.append(a['id'])
    feat_log = []
    for artist in artist_log:
        for track in sp.artist_top_tracks(artist)['tracks']:
            feat_log.append(sp.audio_features(track['id'])[0])
    
    catalog = pd.DataFrame.from_dict(feat_log)
    
    root = pd.DataFrame.from_dict(sp.audio_features(tracks=[track_id]))

    merged_df = root.append(catalog, ignore_index=True)
    
    dropped_df = merged_df.drop(columns=['uri', 'track_href', 'id', 'duration_ms', 'time_signature', 'mode', 'loudness', 'type', 'analysis_url'])
    scaled_df = StandardScaler().fit_transform(dropped_df)
    trans_array = scaled_df.copy()
    trans_array[:,0] = [u*2.4 for u in trans_array[:,0]] # acousticness
    trans_array[:,1] = [((u*u)**0.5)*u for u in trans_array[:,1]] # danceability
    trans_array[:,2] = [u*1.7 for u in trans_array[:,2]] # energy
    trans_array[:,3] = [u*1.4 for u in trans_array[:,3]] # instrumentalness
    trans_array[:,4] = [u*0.9 for u in trans_array[:,4]] # key
    trans_array[:,5] = [u*1.0 for u in trans_array[:,5]] # liveness
    trans_array[:,6] = [u*1.0 for u in trans_array[:,6]] # speechiness
    trans_array[:,7] = [u*1.1 for u in trans_array[:,7]] # tempo
    trans_array[:,8] = [u*2.5 for u in trans_array[:,8]] # valence

    knn = NearestNeighbors()
    knn.fit(trans_array)

    rec = knn.kneighbors(trans_array[[0]], n_neighbors=neighbors+1)

    print('Seed')
    print('ID:     ', root.loc[0,'id'], '\n')
    print('\nRecommendations')

    for n in range(1,neighbors+1):
        if rec[0][0][n] <= max_distance:
            print('ID:      ', merged_df.loc[rec[1][0][n],'id'])
            print('Distance:', rec[0][0][n], '\n')
            print('\n')
            
    if rec[0][0][1] > max_distance:
        print('No matches in catalog')

In [254]:
# Example

dj_rec(song_id, max_distance=6.5, neighbors=3)

Seed
ID:      0gjfdXCV6yrwhP23NVx1Fm 


Recommendations
ID:       0RVIcy8jrmjM9yYfm3HHF4
Distance: 3.4510366756165785 



ID:       3eoXrssdwFYhOlzsJ8sssE
Distance: 4.682273502424689 



ID:       5iTnzXYuEmaJ1ioJ95JC99
Distance: 4.712957518647394 





In [54]:
rec_id = '0RVIcy8jrmjM9yYfm3HHF4'

In [55]:
print(sp.track(song_id)['artists'][0]['name']+ ' - ' 
      + sp.track(song_id)['name']+ ' - ' 
      + sp.track(song_id)['external_urls']['spotify'])

Nite Fleit - Reply All - https://open.spotify.com/track/0gjfdXCV6yrwhP23NVx1Fm


In [56]:
print(sp.track(rec_id)['artists'][0]['name']+ ' - ' 
      + sp.track(rec_id)['name']+ ' - ' 
      + sp.track(rec_id)['external_urls']['spotify'])

Special Request - Arse End of the Moon - https://open.spotify.com/track/0RVIcy8jrmjM9yYfm3HHF4


In [57]:
sp.track(song_id)['artists'][0]['name']

'Nite Fleit'

In [58]:
sp.track(song_id)['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1yVtpTbLBVQ2FykO9CETC5'},
  'href': 'https://api.spotify.com/v1/artists/1yVtpTbLBVQ2FykO9CETC5',
  'id': '1yVtpTbLBVQ2FykO9CETC5',
  'name': 'Nite Fleit',
  'type': 'artist',
  'uri': 'spotify:artist:1yVtpTbLBVQ2FykO9CETC5'}]

In [59]:
sp.track(song_id)['name']

'Reply All'